In [1]:
import sys
import os
import numpy as np
import openmm as mm
from openmm import unit
from openmm.app import PDBFile
from openmm.app.dcdfile import DCDFile
from openmmtools.multistate import MultiStateReporter, ReplicaExchangeAnalyzer
import pdb_numpy
from REST2_initial import run_rest2_hrex
import mdtraj as md

sys.path.append(os.path.abspath('..'))
from src.useful_functions import *


Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************



In [2]:
# Read directory paths
read_dirs_paths('dir_paths.txt', globals())
check_directories(out_dir ,out_trajectories2 ,out_trajectories3 ,out_trajectories4)

Created variables:
inp_dir = /scratch/htc/fsafarov/2cm2_simulation/REST2/input/
out_dir = /scratch/htc/fsafarov/2cm2_simulation/REST2/output/
out_trajectories2 = /scratch/htc/fsafarov/2cm2_simulation/REST2/output/trajectories/openmm_files/
out_trajectories3 = /scratch/htc/fsafarov/2cm2_simulation/REST2/output/trajectories/openmm_files/initial_states/
out_trajectories4 = /scratch/htc/fsafarov/2cm2_simulation/REST2/output/trajectories/openmm_files/final_states/
out_isokann = /scratch/htc/fsafarov/2cm2_simulation/REST2/output/isokann/
out_mokito = /scratch/htc/fsafarov/2cm2_simulation/REST2/output/mokito/
/scratch/htc/fsafarov/2cm2_simulation/REST2/output/ already exists!
/scratch/htc/fsafarov/2cm2_simulation/REST2/output/trajectories/openmm_files/ already exists!
/scratch/htc/fsafarov/2cm2_simulation/REST2/output/trajectories/openmm_files/initial_states/ already exists!
/scratch/htc/fsafarov/2cm2_simulation/REST2/output/trajectories/openmm_files/final_states/ already exists!
 
 


In [3]:
from openmm.app import PDBFile, ForceField
from openmm import unit

# Load your reference system
pdb = PDBFile(inp_dir + "pdbfile_water_1.pdb")
ff  = ForceField("amber14-all.xml","amber14/tip3pfb.xml")
system_ref = ff.createSystem(pdb.topology, nonbondedMethod=mm.app.PME,
                             nonbondedCutoff=1.0*unit.nanometer, constraints=mm.app.HBonds)

# Define solute atom indices (e.g., all protein atoms)
prot_pep_coor = pdb_numpy.Coor(inp_dir + "pdbfile_water_1.pdb")
solute_indices = prot_pep_coor.get_index_select("chain A")


# Ladder (all replicas at 300 K; s from 1.0 down to, say, 0.6)
nrep = 12
s_list = np.linspace(1.0, 0.95, nrep).tolist()
temperatures = [300.0*unit.kelvin]*nrep

run_rest2_hrex(inp_dir + "pdbfile_water_1.pdb", system_ref, solute_indices, temperatures, s_list,
               nsteps=25000, swap_interval=250, timestep=2.0*unit.femtoseconds)


Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w


Potential energy is NaN after 0 attempts of integration with move LangevinDynamicsMove Attempting a restart...


OpenMMException: Particle coordinate is NaN.  For more information, see https://github.com/openmm/openmm/wiki/Frequently-Asked-Questions#nan

In [ ]:
report_path = "rest2_hrex_4.nc"
ref_k = s_list.index(1.0)

an = ReplicaExchangeAnalyzer(report_path)
# This returns an mdtraj.Trajectory for the thermodynamic state index = ref_k
traj = an.read_trajectory(state_index=ref_k)   # one frame per iteration

traj.save_dcd(out_dir + "rest2_ref_s1.0.dcd")
